In [1]:
list_country = [
    'BR', #brésil
    'CO', #colombie
    'PE', #peru
    'CM', #cameroun
    'CF', #congo
    'CG', #rdc
    'GB', #gabon
    'UG', #Ouganda
    'GH', #ghana
    'IV', #côte d'Ivoire
    'ID', #indonésie
    'MY', #malaisie
]

In [2]:
country_code = list_country[3] 

In [3]:
import geemap
import ee

ee.Initialize()

Map = geemap.Map()
country = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_co', country_code))
Map.addLayer(country, {}, 'country_code')
Map.centerObject(country)
Map

Map(center=[5.683679386840729, 12.738353454424], controls=(WidgetControl(options=['position'], widget=HBox(chi…

In [4]:
from sepal_ui import widgetFactory as wf

output = wf.OutputWidget()
output

Alert(children=['Click it'], class_='mt-5', text=True, type='info')

In [5]:
from scripts import zonal_computation as zc

#launch the process 
raw_data = '/home/prambaud/zonal_results/CM/CM_raw_data.txt'

In [6]:
import pandas as pd
from scripts import zonal_computation as zc

df = pd.read_csv(raw_data, sep=' ', header=None)
names = ['code', 'total'] + ['tc{:02}'.format(i) for i in range (df.shape[1]-2)]
df.columns = names

#ajouter les colonnes maquantes 
for i in range(df.shape[1]-2, 100+1):
    df['tc{:02}'.format(i)] = [0 for j in range(len(df))]
    
df['zone'] = [zc.get_ecozones()[code] for code in df['code']]
    
[zc.get_ecozones()[code] for code in df['code']]

['Tropical rainforest',
 'Tropical moist forest',
 'Tropical dry forest',
 'Tropical shrubland',
 'Tropical mountain system',
 'Water']

In [7]:
list_zones = zc.get_ecozones()
for ecozone in list_zones:
    print(df.loc[df['code'] == ecozone])

Empty DataFrame
Columns: [code, total, tc00, tc01, tc02, tc03, tc04, tc05, tc06, tc07, tc08, tc09, tc10, tc11, tc12, tc13, tc14, tc15, tc16, tc17, tc18, tc19, tc20, tc21, tc22, tc23, tc24, tc25, tc26, tc27, tc28, tc29, tc30, tc31, tc32, tc33, tc34, tc35, tc36, tc37, tc38, tc39, tc40, tc41, tc42, tc43, tc44, tc45, tc46, tc47, tc48, tc49, tc50, tc51, tc52, tc53, tc54, tc55, tc56, tc57, tc58, tc59, tc60, tc61, tc62, tc63, tc64, tc65, tc66, tc67, tc68, tc69, tc70, tc71, tc72, tc73, tc74, tc75, tc76, tc77, tc78, tc79, tc80, tc81, tc82, tc83, tc84, tc85, tc86, tc87, tc88, tc89, tc90, tc91, tc92, tc93, tc94, tc95, tc96, tc97, ...]
Index: []

[0 rows x 104 columns]
Empty DataFrame
Columns: [code, total, tc00, tc01, tc02, tc03, tc04, tc05, tc06, tc07, tc08, tc09, tc10, tc11, tc12, tc13, tc14, tc15, tc16, tc17, tc18, tc19, tc20, tc21, tc22, tc23, tc24, tc25, tc26, tc27, tc28, tc29, tc30, tc31, tc32, tc33, tc34, tc35, tc36, tc37, tc38, tc39, tc40, tc41, tc42, tc43, tc44, tc45, tc46, tc47, tc48, t

In [9]:
values = df.filter(items=['zone']+['tc{:02}'.format(i) for i in range(100)])
values
list_zones = [zc.get_ecozones()[code] for code in df['code']]
for ecozone in list_zones:
    print(ecozone)
    tmp = df.loc[df['zone'] == ecozone]
    values = tmp.filter(items=['tc{:02}'.format(i) for i in range(100)])
    
    print(values.max(axis=1).array[0])
    
    break

Tropical rainforest
22582341
